# Kaggle Competition

## 2. Modeling

In [ ]:
from __future__ import unicode_literals, division

import numpy as np
import pandas as pd

# Plotly
from plotly.offline import init_notebook_mode, iplot
import cufflinks as cf

init_notebook_mode()
cf.go_offline()

# Jupyter Notebook Options
import warnings

from IPython.display import HTML
from IPython.display import display as prnt

warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

#### Utility Functions

In [ ]:
def table(df,replace_match="",replace_str=""):
    return IPython.display.display(HTML(df.to_html().replace('<table border="1" class="dataframe">','<table class="table table-striped table-hover">').replace(replace_match,replace_str)))

### Load the Data

In [ ]:
DATA_DIR = '../data/'
TRAIN_FILE = DATA_DIR + 'train.csv'
VALIDATION_FILE = DATA_DIR + 'test.csv'
df = pd.read_csv(TRAIN_FILE)

In [ ]:
pd.read_csv(TRAIN_FILE).shape

In [ ]:
pd.read_csv(VALIDATION_FILE).shape

#### Training / Test Split

In [ ]:
df.ix[:,'casual':].head()

In [ ]:
def get_train_data():
    # Loads the training data, but splits the y from the X
    dfx = pd.read_csv(TRAIN_FILE)
    return dfx.iloc[:, 0:9], dfx.iloc[:,-1]

In [ ]:
A = get_train_data()

In [ ]:
X, y = get_train_data()

### Scoring Method

$ϵ$ is the RMSLE value (score) $n$ is the total number of observations in the (public/private) data set, $p_i$ is your prediction, and $a_i$ is the actual response for $i$. $log(x)$ is the natural logarithm of $x$

$$\epsilon = \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 }$$

RMSLE penalizes an under-predicted estimate greater than an over-predicted estimate

In [ ]:
np.log(1)

In [ ]:
from sklearn.metrics import make_scorer

# First, we should set up some sort of testing framework, so that we can benchmark our progress as we go
# The evaluation metric is Root mean squared logarithmic error.
def rmsele(actual, pred):
    """
    Given a column of predictions and a column of actuals, calculate the RMSELE
    """
    squared_errors = (np.log(pred + 1) - np.log(actual + 1)) ** 2
    mean_squared = np.sum(squared_errors) / len(squared_errors)
    return np.sqrt(mean_squared)

# This helper function will make a callable that we can use in cross_val_score
rmsele_scorer = make_scorer(rmsele, greater_is_better=False)

### Baseline Model

In [ ]:
len(df)

In [ ]:
expected_value = df['count'].mean()

In [ ]:
# [expected_value] * len(df)

In [ ]:
yhat = np.array([expected_value] * len(df))

rmsele(y, yhat)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.cross_validation import cross_val_score

# Lets just train a basic model so that we can test if our scoring and
# cross validation framework works well. We'll use a Ridge regression,
# which is a form of linear regression
X, y = get_train_data()

In [ ]:
# Subset the X to just use temp, atemp, and workingday
Xhat = X[['temp', 'atemp', 'humidity']]

ridge_estimator = Ridge(normalize=True)

scores = cross_val_score(ridge_estimator, Xhat, y, scoring=rmsele_scorer, cv=5)

abs(scores.mean())

In [ ]:
rmsele(y, yhat) / abs(scores.mean())

In [ ]:
print scores
scores.std()

#### Sanity Checking

In [ ]:
Xhat = X[['temp', 'atemp', 'humidity']]

ridge_estimator = Ridge()

cross_val_score(ridge_estimator, Xhat, y, scoring=rmsele_scorer, cv=5).mean()

Why is that the case? Higher/Lower for which model?

* Values
* Coefficients
* Penalisation
* Bias

In [ ]:
from plotly.tools import FigureFactory as FF

iplot(FF.create_scatterplotmatrix(df[['temp', 'atemp', 'humidity','count']].sample(2000),width=920,height=700));

### CrossValidation

In [ ]:
# Fill in some of the parameters on cross_val_score
def perform_cv(estimator, X, y):
    return cross_val_score(estimator, X, y, scoring=rmsele_scorer, cv=5)

In [ ]:
ridge_estimator.fit(Xhat, y)

In [ ]:
Ridge??

In [ ]:
class FlooredRidge(Ridge):
    def __init__(self, *args, **kwargs):
        super(FlooredRidge, self).__init__(*args,**kwargs)
    
    def predict(self, X):        
        pred = super(FlooredRidge, self).predict(X) 
        pred[pred < 0] = 0
        return pred

In [ ]:
X[ridge_estimator.predict(Xhat) > 0].temp.iplot(kind='hist',bins=25,dimensions=(900,500))

In [ ]:
X[ridge_estimator.predict(Xhat) < 0].temp.iplot(kind='hist',bins=25,dimensions=(900,500))

### Grid Search

In [ ]:
# len(range(1,26,2)) * 10 * 30

In [ ]:
np.logspace(0, 3.5, 10)

In [ ]:
from sklearn.grid_search import GridSearchCV

# Try a simple grid search with the estimator
parameters = {'alpha': np.logspace(0, 3.5, 10),
              'normalize' : [True,False]}
grid = GridSearchCV(ridge_estimator, parameters, scoring=rmsele_scorer, cv=5)

Xhat = X[['temp', 'atemp', 'humidity']
# Xhat = X[['temp', 'humidity']

grid.fit(Xhat, y)
grid.grid_scores_

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_estimator_.coef_

In [ ]:
df_gridscore = pd.concat([pd.DataFrame([x.mean_validation_score for x in grid.grid_scores_]), pd.DataFrame([x.parameters for x in grid.grid_scores_])],axis=1)
df_gridscore.set_index(['alpha','normalize']).unstack().iplot()

In [ ]:
# And for grid_search
def perform_grid_search(estimator, parameters, X, y):
    grid_search = GridSearchCV(estimator, parameters, scoring=rmsele_scorer, cv=5)
    grid_search.fit(X, y)
    return grid_search

## Transform Data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df

In [ ]:
normalize??

In [ ]:
Xhat = X[['temp', 'atemp', 'humidity']]
normalize = StandardScaler()
normalize.fit(Xhat)

In [ ]:
((df[['temp', 'atemp', 'humidity']] - df[['temp', 'atemp', 'humidity']].mean()) / df[['temp', 'atemp', 'humidity']].std()).head(5)

In [ ]:
print normalize.scale_
print normalize.mean_
(Xhat - Xhat.mean()) / Xhat.std()
((Xhat - normalize.mean_) / normalize.scale_).head()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Now lets move on to the actual transformation of the inputs
# First, not every estimator we'll use will have the "normalize" keyword
# So let's break it out into a transformer, so that we have better control over it
ridge_estimator = Ridge()
normalize = StandardScaler()

Xhat = X[['temp', 'atemp', 'humidity']]
Xhat = normalize.fit_transform(Xhat)

scores = perform_cv(ridge_estimator, Xhat, y)
scores.mean()

### Pipeline

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion

# Now we have the beginnings of a multi-step pipeline
# Scikit lets you wrap each of these steps into a Pipeline object,
# so you just have to run fit / predict once
# instead of manually feeding the data from one transformer to the next
normalize = StandardScaler()
ridge_estimator = Ridge()

pipeline = Pipeline([
        ('normalize', normalize),
        ('ridge', ridge_estimator)
    ])

Xhat = X[['temp', 'atemp', 'humidity']]
perform_cv(pipeline, Xhat, y).mean()

In [ ]:
# What if you wanted to build your custom transformer?

from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

class DivideByTwo(TransformerMixin):
        
    def transform(self, X, *_):
        return result / 2
    
    def fit(self, *_):
        return self
        

In [ ]:
# Additionally, you can perform grid search over all of the steps of the pipeline
# So you don't have to tune each step manually
# The pipeline exposes the underlying steps' parameters like so:
# ridge__alpha, and normalize__with_mean

normalize = StandardScaler()
ridge_estimator = Ridge()

parameters = {'ridge__alpha': np.logspace(0, 3, 10)}

Xhat = X[['temp', 'atemp', 'humidity']]

pipeline = Pipeline([('normalize', normalize), ('ridge', ridge_estimator)])

grid = GridSearchCV(pipeline, parameters, scoring=rmsele_scorer, cv=5)
grid.fit(Xhat, y)

grid.grid_scores_

In [ ]:
def plot_grid_scores(grid, parameter='ridge__alpha', log_scale=True):
    idx = map(lambda x: x[0][parameter], grid.grid_scores_)
    fig = pd.DataFrame(map(lambda x: x[1], grid.grid_scores_),index=idx).iplot(dimensions=(900,500), asFigure=True)
    if log_scale:
        fig['layout']['xaxis1'].update({'type':'log'})
    iplot(fig)

In [ ]:
plot_grid_scores(grid)

### Feature Engineering

#### Encoding Dummy Variables

In [ ]:
df.season.sample(2000).iplot(kind='hist', dimensions=(900,400))

In [ ]:
df.sample(2000).iplot(x='count',y='season',mode='markers',dimensions=(900,400))

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer

# Lets move on to including more features in our model
# We probably want to use a factor like Season in our model, but it's
# a categorical feature, and we'll need to convert it to a series of booleans

one_hot = OneHotEncoder()
# season = one_hot.fit_transform(X['season'].reshape(X.shape[0], 1)).toarray()
season = one_hot.fit_transform(X[['season']]).toarray()

In [ ]:
one_hot.fit_transform(X[['season']])

In [ ]:
one_hot.fit_transform(X[['season']]).toarray()

In [ ]:
# We then have to join this with the other variables
ridge_estimator = Ridge()

normalize = StandardScaler()
pipeline = Pipeline([('normalize', normalize), ('ridge', ridge_estimator)])

season = one_hot.fit_transform(X[['season']]).toarray()
Xhat = np.hstack([X[['temp', 'atemp', 'humidity']], season])

perform_cv(pipeline, Xhat, y).mean()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# There's a faster way of doing this with the argument 'categorical_features', but this is the basic
# way through which you would extend the functionality if SciKit Learn

class ToArray(BaseEstimator, TransformerMixin):
    # We need this because OneHotEncoder returns a sparse array, and normalize requires a non-sparse array
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.toarray()
        
Xhat = X[['season', 'weather', 'temp', 'atemp', 'humidity']]

# I think it needs to be 5 here, because it assumes that '0' is a possible value for an int datatype
# Should probably specify the data types in read_csv

one_hot = OneHotEncoder(n_values=[5,5], categorical_features=[0, 1])

desparse = ToArray()
normalize = StandardScaler()
ridge_estimator = Ridge()

pipeline = Pipeline([('onehot', one_hot),
                     ('desparse', desparse),
                     ('normalize', normalize),
                     ('ridge', ridge_estimator)
                    ])
perform_cv(pipeline, Xhat, y).mean()

#### Selective Normalisation

In [ ]:
StandardScaler??

In [ ]:
# OK, so now we've got a pipeline that does one-hot encoding of two categorical variables
# and then normalizes the variables
# But actually we're not supposed to normalize the the dummy variables.
# So we need some way of only normalizing non-dummy variables

# Oops, actually the CV splitting converts the Pandas DF to an array, so we can't rely
# on the normalize having the proper column names
class SelectiveNormalize(StandardScaler):
    def __init__(self, cols, copy=True, with_mean=True, with_std=True):
        self.cols = cols
        super(SelectiveNormalize, self).__init__(copy, with_mean, with_std)
    
    def fit(self, X, y=None):        
        subset = X.iloc[:, self.cols]
        return super(SelectiveNormalize, self).fit(subset, y)
        
    def transform(self, X):
        subset = X.iloc[:, self.cols]
        normalized = super(SelectiveNormalize, self).transform(subset)
        others = [col for col in range(X.shape[1]) if col not in self.cols]
        res = np.hstack([X.iloc[:, others],normalized])
        return res

Xhat = X[['season', 'weather', 'temp', 'atemp', 'humidity']]

one_hot = OneHotEncoder(n_values=[5, 5], categorical_features=[0, 1])

normalize = SelectiveNormalize([2, 3, 4])
desparse = ToArray()
ridge_estimator = Ridge()

pipeline = Pipeline([('normalize', normalize), ('onehot', one_hot), ('desparse', desparse), ('ridge', ridge_estimator)])
perform_cv(pipeline, Xhat, y).mean()

#### Datetime

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Lets try tackling the date column now.The time of day is probably really important
# So we need some way of extracting the hour
# We'll use a FeatureUnion to do this, to demonstrate the functionality

def get_train_data():
    # Loads the training data, but splits the y from the X
    df = pd.read_csv(TRAIN_FILE, parse_dates=['datetime'])
    return df.iloc[:, 0:9], df.iloc[:,-1]

In [ ]:
class SelectColumns(BaseEstimator, TransformerMixin):
    """
    Passes on a subset of columns from an input ndarray
    """
    def __init__(self, cols):
        self.cols = cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.iloc[:, self.cols]

In [ ]:
class ExtractHour(BaseEstimator, TransformerMixin):
    """
    Extracts hour from a datetime series
    """
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        res = np.zeros(X.shape)
        for xx in xrange(X.shape[0]):
            res[xx] = X.iloc[xx, 0].hour
        return res.reshape(res.shape[0], 1)

In [ ]:
class CastType(BaseEstimator, TransformerMixin):
    def __init__(self, cast_to):
        self.cast_to = cast_to
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.astype(self.cast_to)

X, y = get_train_data()

# Reminder of the columns:
# ['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']

select_date = SelectColumns([0])
select_others = SelectColumns(range(1, 9))

cast_float = CastType(np.float64)
one_hot = OneHotEncoder(n_values=[5, 5], categorical_features=[0, 3])
get_hour = ExtractHour()
normalize = SelectiveNormalize(range(2, 8))
desparse = ToArray()

In [ ]:
ridge_estimator = Ridge()

hour_feature = Pipeline([
        ('select_date', select_date),
        ('get_hour', get_hour)
    ])

other_features = Pipeline([
        ('select_others', select_others),
        ('cast_float', cast_float),
        ('onehot', one_hot),
        ('desparse', desparse)
    ])

join_features = FeatureUnion([
        ('hour', hour_feature),
        ('others', other_features)
    ])

predict = Pipeline([
        ('featurize', join_features),
        ('estimator', ridge_estimator)])

scores = perform_cv(predict, X, y)
scores.mean()

In [ ]:
rf_estimator = RandomForestRegressor(n_estimators=20)

predict = Pipeline([
        ('featurize', join_features),
        ('estimator', rf_estimator)])

scores = perform_cv(predict, X, y)
abs(scores.mean())


### Hyperparameter Finetuning

In [ ]:
for attr in sorted(predict.get_params().keys()):
    print attr

In [ ]:
rf_estimator = RandomForestRegressor(n_estimators=1)

predict = Pipeline([
        ('featurize', join_features),
        ('estimator', rf_estimator)])

params = {
    'estimator__max_depth' : [2,3,5,8,10,20,100],
    'estimator__min_samples_split' : range(2,10)
}

grid = perform_grid_search(predict, params, X, y)

In [ ]:
grid.best_score_

In [ ]:
plot_grid_scores(grid,'estimator__max_depth')

In [ ]:
plot_grid_scores(grid,'estimator__min_samples_split')

## Submission

In [ ]:
def make_submission(df_test, prediction, filename='submission.csv'):
    with open(filename, 'w') as f:
        f.write('datetime,count\n')
        submission_strings = df_test.reset_index()['datetime'] + ',' + prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n')

# make_submission(df_test, prediction, 'submission.csv')